In [1]:
import pandas as pd
import os
import numpy as np
import fiona
from shapely.geometry import Polygon
from shapely.geometry import mapping
import folium
import geopandas as gpd
import random
import ast
# workspace_directory = r'C:\Users\trygg\Documents\Master_project' #HOME
workspace_directory = r'C:\Users\tryggvisi\Documents\my-awesome-masters-project' #WORK
os.chdir(workspace_directory)# Change the current working directory to the specified workspace
#gee_env

In [3]:
import ast

def string_to_list(input_string):
    """
    Converts a string representation of a list into an actual list.

    Parameters:
    input_string (str): The string representation of the list.

    Returns:
    list: The converted list.
    """
    result_list = ast.literal_eval(input_string)
    return result_list

#Ná í allar unique dagsetningar
def Unique_dates(dictionary):
    """
    Returns a list of unique dates from a dictionary.

    Parameters:
    dictionary (list): A list of dictionaries containing date information.

    Returns:
    list: A sorted list of unique dates.
    """
    dates = []
    for i in range(len(dictionary)):
        dates.append(dictionary[i]['date'])
    unique_dates = list(dict.fromkeys(dates).keys())
    unique_dates.sort()
    return unique_dates

def flip_coords(footprint_str):
    """
    Flips the coordinates in a given footprint string.

    Parameters:
    footprint_str (str): The footprint string containing coordinate pairs.

    Returns:
    list: A list of coordinate pairs with flipped latitudes and longitudes.
    """
    coordinate_pairs = []
    for pair_str in footprint_str:
        temp = [round(float(pair_str[1]),2), round(float(pair_str[0]),2)]
        coordinate_pairs.append(temp)  # Swap lat and lon to follow (lon, lat) convention
    pairs_within_pairs = [coordinate_pairs]
    return pairs_within_pairs

def color_generator(teljari = 0):
    """
    Generates a random color.

    Returns:
    str: A random color in hexadecimal format.
    """
    color = ['black', 'red', 'blue']
    return color[teljari]

In [5]:
#allar skrár sem þarf til að gera kort af study area, granules, punkum innalands og triangle corners

#Skrá með öllum SLC granules
path_granuleID = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\list_of_slc_granules_2023.csv" #with 2023 data
# path_granuleID = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_granule_id_slc.csv"
gdf_path_granuleID = gpd.read_file(path_granuleID)
# gdf2 = gdf2.drop(gdf2.columns[-1], axis=1)
ordabok_granule_id_slc = gdf_path_granuleID.to_dict(orient = 'index')

#allir triangle corners í rannsókninni með celcius
path_cornerTriangles =  r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Data\Vindefjallen_data\Vindefjallen_cleaning\vindefjallen_corner_coords_celcius.csv"
gdf_path_cornerTriangles = gpd.read_file(path_cornerTriangles)
vindefjallen_triangleCorners_numpy_celcius = gdf_path_cornerTriangles.to_numpy()

#Skrá með öllum mælipunktum innan SLC granule
path_granule_every_point = r"C:\Users\tryggvisi\Documents\my-awesome-masters-project\Data\Vindefjallen_data\Vindefjallen_cleaning\slc_granule_every_point.csv"
gdf_path_granule_every_point = gpd.read_file(path_granule_every_point)

#Shp af study area Vindefjallen
vector_shp = r"Data\Vindefjallen_data\outputs\shapefiles\study_area_4326.shp"
# vector_shp = r"Data\Vindefjallen_data\outputs\shapefiles\Vindefjallen_extent_updated4326.shp"
gdf_vector = gpd.read_file(vector_shp)

path_final_meas = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\1final_measurement_values_v1.csv"
gdf_final_points = pd.read_csv(path_final_meas)
#change column names x_4326 and y_4326 to x_coord and y_coord
#drop first 4 columns
gdf_final_points = gdf_final_points.drop(gdf_final_points.columns[0:4], axis = 1)
gdf_final_points = gdf_final_points.rename(columns = {'x_4326':'x_coord', 'y_4326':'y_coord', 'Corner': 'corner', 'SnowDepth': 'snow_depth'})

#for consistency, gdf_path_granule_every_point becomes gdf_final_points
gdf_path_granule_every_point = gdf_final_points


In [11]:
gdf_path_granule_every_point

,Triangle,corner,TriangleCorner,date,snow_depth,SnowDepth_Noplus,temperature,x_coord,y_coord,x_3006,y_3006,MiddleTime,mask.tif,angle.tif,VH.tif,VV.tif,point_VH,point_VV,points_angle
0,AC-42,C,AC-42C,2017-02-21,110,110,-5,15.685047,65.890430,531224.7772,7307867.528,10:50:00,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.021,0.093,46.351
1,AC-31,A,AC-31A,2017-02-26,20,20,-8,15.595766,65.984498,527055.8412,7318310.670,13:52:30,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.013,0.088,44.248
2,AC-31,C,AC-31C,2017-02-26,65,65,-8,15.683463,65.987723,531034.4515,7318710.707,13:52:30,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.017,0.103,43.178
3,AC-38,B,AC-38B,2017-02-26,75,75,-10,16.066011,65.929045,548515.1091,7312413.359,08:55:00,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.012,0.080,35.335
4,AC-38,C,AC-38C,2017-02-26,75,75,-10,16.116060,65.899573,550851.1264,7309168.337,08:55:00,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.014,0.108,32.779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,AC-30,A,AC-30A,2023-03-25,45,45,-10,16.323925,66.073454,559911.9498,7328730.193,11:32:30,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.008,0.099,32.286
333,AC-30,C,AC-30C,2023-03-25,60,60,-10,16.409487,66.082425,563760.9059,7329814.416,11:32:30,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.004,0.063,28.861
334,AC-28,A,AC-28A,2023-03-26,130,130,-8,15.849172,66.153233,538308.6585,7337249.602,12:52:30,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.013,0.082,34.289
335,AC-28,C,AC-28C,2023-03-26,45,45,-8,15.906029,66.125729,540917.8704,7334220.113,12:52:30,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,C:/Users/tryggvisi/Documents/my-awesome-master...,0.009,0.093,31.515


In [16]:
# create a folium map
m = folium.Map(location=[66.0, 23.0], zoom_start=5)
feature_group = folium.FeatureGroup(name=f"Study area", show=False)  # Create feature group which will be added to later
folium.GeoJson(gdf_vector).add_to(feature_group)  # Add the vector file to the feature group

folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)
feature_group.add_to(m)
feature_group = folium.FeatureGroup(name=f"Vindefjällen measurement points", show=False)
unique_dates = Unique_dates(ordabok_granule_id_slc)
###################################################################

corner_colors = {
    "A": "red",
    "B": "blue",
    "C": "green"
}


for idx, row in gdf_path_granule_every_point.iterrows():
    corner = row["corner"][-1:]
    color = corner_colors.get(corner, "black")
    
    folium.CircleMarker(
        location=[row['y_coord'], row['x_coord']],
        color=color,
        fill_color=color,
        radius=5,
        fill=True,
        fill_opacity=1,
        popup=folium.Popup(
            f"Corner: {row['TriangleCorner']}<br>",
            max_width=500,
        ),
        tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
    ).add_to(feature_group)
    feature_group.add_to(m)

###################################################################
    
#New version of code
for dagsetning in unique_dates:
    if dagsetning == '2017-02-21':
        feature_group = folium.FeatureGroup(name=f"Group for {dagsetning}", show=True)
    elif dagsetning != '2017-02-21':
        feature_group = folium.FeatureGroup(name=f"Group for {dagsetning}", show=False)
    teljari = 0
    for i in ordabok_granule_id_slc:
        if ordabok_granule_id_slc[i]["date"] == dagsetning:
            
            polygon = flip_coords(string_to_list(ordabok_granule_id_slc[i]["polygon"]))
            folium.Polygon(
                locations=polygon,
                color=color_generator(teljari),
                fill=True,
                fill_color=color_generator(teljari),
                fill_opacity=0.1,
                popup=folium.Popup(
                    f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}<br>"
                    f"PathNr: {ordabok_granule_id_slc[i]['pathNr']}<br>"
                    f"FrameNr: {ordabok_granule_id_slc[i]['frameNr']}<br>"
                    f"Date: {ordabok_granule_id_slc[i]['date']}<br>"
                    f"Start Time: {ordabok_granule_id_slc[i]['startTime']}<br>"
                    f"Direction: {ordabok_granule_id_slc[i]['orbitProp']}",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(
                    text=f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}"
                ),
            ).add_to(feature_group)
            feature_group.add_to(m)
            teljari += 1
    for row in vindefjallen_triangleCorners_numpy_celcius:
        if row[3] == dagsetning:
            folium.CircleMarker(
                location=[row[7], row[6]],
                color="black",
                fill_color="black",
                radius=3,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row[0]}<br>"
                    f"Corner: {row[5]}<br>"
                    f"Snow Depth: {row[4]} cm<br>"
                    f"Date: {row[3]}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"Outside footprint<br>{row[5]}"),
            ).add_to(feature_group)
            feature_group.add_to(m)
    for idx, row in gdf_path_granule_every_point.iterrows():
        if row["date"] == dagsetning:
            folium.CircleMarker(
                location=[row['y_coord'], row['x_coord']],
                color="red",
                fill_color="red",
                radius=5,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"Triangle Corner: {row['TriangleCorner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>"
                    f"VV: {row['point_VH']}<br>"
                    f"VH: {row['point_VV']}<br>"
                    f"Angle: {row['points_angle']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['TriangleCorner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)
    
# unique_dates.pop(0)
folium.LayerControl().add_to(m)

m


In [ ]:
#OLd and outdated code
#add only the first granule to the map


unique_dates = Unique_dates(ordabok_granule_id_slc)
for dagsetning in unique_dates:
    feature_group = folium.FeatureGroup(name=f"Group for {dagsetning}", show=True)
    teljari = 0
    for i in ordabok_granule_id_slc:
        if ordabok_granule_id_slc[i]["date"] == dagsetning:
            
            polygon = flip_coords(string_to_list(ordabok_granule_id_slc[i]["polygon"]))
            folium.Polygon(
                locations=polygon,
                color=color_generator(teljari),
                fill=True,
                fill_color=color_generator(teljari),
                fill_opacity=0.1,
                popup=folium.Popup(
                    f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}<br>"
                    f"PathNr: {ordabok_granule_id_slc[i]['pathNr']}<br>"
                    f"FrameNr: {ordabok_granule_id_slc[i]['frameNr']}<br>"
                    f"Date: {ordabok_granule_id_slc[i]['date']}<br>"
                    f"Start Time: {ordabok_granule_id_slc[i]['startTime']}<br>"
                    f"Direction: {ordabok_granule_id_slc[i]['orbitProp']}",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(
                    text=f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}"
                ),
            ).add_to(feature_group)
            feature_group.add_to(m)
            teljari += 1
    for row in vindefjallen_triangleCorners_numpy_celcius:
        if row[3] == dagsetning:
            folium.CircleMarker(
                location=[row[7], row[6]],
                color="black",
                fill_color="black",
                radius=3,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row[0]}<br>"
                    f"Corner: {row[5]}<br>"
                    f"Snow Depth: {row[4]} cm<br>"
                    f"Date: {row[3]}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"Outside footprint<br>{row[5]}"),
            ).add_to(feature_group)
            feature_group.add_to(m)
    for idx, row in gdf_path_granule_every_point.iterrows():
        if row["date"] == dagsetning:
            folium.CircleMarker(
                location=[row['y_coord'], row['x_coord']],
                color="red",
                fill_color="red",
                radius=5,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"Corner: {row['corner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)
    break
# unique_dates.pop(0)

# Add every granule and measurement to the map, and also the measurement points that did not correspond to a granule

unique_dates1 = Unique_dates(ordabok_granule_id_slc)
unique_dates = unique_dates1[1:]

for dagsetning in unique_dates:
    feature_group = folium.FeatureGroup(name=f"Group for {dagsetning}", show=False)
    teljari = 0
    for i in ordabok_granule_id_slc:  # every granule
        if ordabok_granule_id_slc[i]["date"] == dagsetning:
            polygon = flip_coords(string_to_list(ordabok_granule_id_slc[i]["polygon"]))
            folium.Polygon(
                locations=polygon,
                color=color_generator(teljari),
                fill=True,
                fill_color=color_generator(teljari),
                fill_opacity=0.1,
                popup=folium.Popup(
                    f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}<br>"
                    f"PathNr: {ordabok_granule_id_slc[i]['pathNr']}<br>"
                    f"FrameNr: {ordabok_granule_id_slc[i]['frameNr']}<br>"
                    f"Date: {ordabok_granule_id_slc[i]['date']}<br>"
                    f"Start Time: {ordabok_granule_id_slc[i]['startTime']}<br>"
                    f"Direction: {ordabok_granule_id_slc[i]['orbitProp']}",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(
                    text=f"GranuleID: {ordabok_granule_id_slc[i]['granule_ID']}"
                ),
            ).add_to(feature_group)
            feature_group.add_to(m)
            # print(teljari)
            teljari += 1
    for row in vindefjallen_triangleCorners_numpy_celcius:  # every measurement point, even if it did not correspond to a granule
        if row[3] == dagsetning:
            folium.CircleMarker(
                location=[row[7], row[6]],
                color="black",
                fill_color="black",
                radius=3,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"ID: {row[0]}<br>"
                    f"Corner: {row[5]}<br>"
                    f"Snow Depth: {row[4]} cm<br>"
                    f"Date: {row[3]}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"Outside footprint<br>{row[5]}"),
            ).add_to(feature_group)
            feature_group.add_to(m)
    for idx, row in gdf_path_granule_every_point.iterrows():  # every measurement point that corresponded to a granule
        if row["date"] == dagsetning:
            color = "red"  # Change the color here
            folium.CircleMarker(
                location=[row['y_coord'], row['x_coord']],
                color=color,
                fill_color=color,
                radius=5,
                fill=True,
                fill_opacity=1,
                popup=folium.Popup(
                    f"Corner: {row['corner']}<br>"
                    f"Snow Depth: {row['snow_depth']} cm<br>"
                    f"Date: {row['date']}<br>",
                    max_width=500,
                ),
                tooltip=folium.Tooltip(text=f"{row['corner']}<br>"),
            ).add_to(feature_group)
            feature_group.add_to(m)

In [17]:
m.save('final_measurements_2023_ESRI_med_values.html')
